In [4]:
from datasets.wider_global_dataset import build_wider_dataloader
from datasets.text_test_datasets import build_text_test_loader
from datasets.image_test_datasets import build_image_test_loader
from manager import Manager, build_graph_optimizer
from evaluators.global_evaluator import GlobalEvaluator
from evaluators.np_evaluator import NPEvaluator

from loggers.logger import Logger
from tqdm import tqdm as tqdm
import os
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch
from configs.args import load_arg_parser

#------------------
## args
#------------------
parser = load_arg_parser()
cfg = parser.parse_args("")
root = cfg.data_root

# data path
cfg.anno_path = os.path.join(root, cfg.anno_path)
cfg.img_dir = os.path.join(root, cfg.img_dir)
cfg.val_anno_path = os.path.join(root, cfg.val_anno_path)
cfg.val_img_dir = os.path.join(root, cfg.val_img_dir)
cfg.gt_file_fn = os.path.join(root, cfg.gt_file_fn)

# meta data path
cfg.cheap_candidate_fn = os.path.join(root, cfg.cheap_candidate_fn)
cfg.vocab_path = os.path.join(root, cfg.vocab_path)

# sys path

cfg.dim = (384,128)
cfg.exp_name = "dist_fn_{}_imgbb_{}_capbb_{}_embed_size_{}_batch_{}_lr_{}_captype_{}_img_meltlayer_{}_cos_margin_{}_np_{}".format(
    cfg.dist_fn_opt,
    cfg.img_backbone_opt,
    cfg.cap_backbone_opt,
    cfg.embed_size,
    cfg.batch_size,
    cfg.lr,
    cfg.cap_embed_type,
    cfg.image_melt_layer,
    cfg.cos_margin,
    cfg.np)

cfg.np=False
cfg.model_path = os.path.join("/shared/rsaas/aiyucui2/wider_person", cfg.model_path, cfg.exp_name)
cfg.output_path = os.path.join("/shared/rsaas/aiyucui2/wider_person", cfg.output_path, cfg.exp_name)

if not os.path.exists(cfg.model_path):
    os.mkdir(cfg.model_path)
if not os.path.exists(cfg.output_path):
    os.mkdir(cfg.output_path)

logger = Logger(os.path.join(cfg.output_path, "log.txt"))
logger.log(str(cfg))




Namespace(anno_path='/shared/rsaas/aiyucui2/wider_person/wider/train/train_anns_train.json', batch_size=64, cap_backbone_opt='bigru', cap_embed_type='sent', cheap_candidate_fn='/shared/rsaas/aiyucui2/wider_person/wider_graph/summer_best_val1_top100.pkl', cheap_eval=True, ckpt_freq=10, cos_margin=0.5, data_root='/shared/rsaas/aiyucui2/wider_person/', debug=True, dim=(384, 128), dist_fn_opt='euclidean', embed_size=2048, exp_name='dist_fn_euclidean_imgbb_resnet50_capbb_bigru_embed_size_2048_batch_64_lr_0.0001_captype_sent_img_meltlayer_1_cos_margin_0.5_np_True', experiment_name='default', gt_file_fn='/shared/rsaas/aiyucui2/wider_person/wider/val1/val_label.json', image_melt_layer=1, img_backbone_opt='resnet50', img_dir='/shared/rsaas/aiyucui2/wider_person/wider/train/img', img_num_cut=1, load_ckpt_fn='0', load_model_path='starter_bert_resnet50_2048.pt', lr=0.0001, mode='train', model_path='/shared/rsaas/aiyucui2/wider_person/checkpoints/reID/dist_fn_euclidean_imgbb_resnet50_capbb_bigru_em

In [5]:
#------------------
## agents
#------------------
# Data loaders
train_loader = build_wider_dataloader(cfg)
cfg.num_ids = len(train_loader.dataset.person2label.values())
test_text_loader = build_text_test_loader(cfg) 
test_image_loader = build_image_test_loader(cfg) 

# Evaluators 
Evaluator = NPEvaluator if cfg.np else GlobalEvaluator

evaluator = Evaluator(img_loader=test_image_loader, 
                          cap_loader=test_text_loader, 
                          gt_file_path=cfg.gt_file_fn,
                          embed_size=cfg.embed_size,
                          logger=logger,
                          dist_fn_opt="euclidean")
cos_evaluator = Evaluator(img_loader=test_image_loader, 
                          cap_loader=test_text_loader, 
                          gt_file_path=cfg.gt_file_fn,
                          embed_size=cfg.embed_size,
                          logger=logger,
                          dist_fn_opt="cosine")
# Trainer
manager = Manager(cfg, logger)




[ds] load annotations from /shared/rsaas/aiyucui2/wider_person/wider/train/train_anns_train.json
size of dataset: 1000
[Trainer][init] initialize fresh model.
[Trainer][init] model initialized.
[Trainer][init] criterion initialized.


In [6]:
#------------------
## Train
#------------------
# Stage 1 (ID)
logger.log("======== [Stage 1] ============")
manager.melt_img_layer(num_layer_to_melt=1)
param_to_optimize = build_graph_optimizer([manager.model, manager.id_cls])
optimizer = optim.Adam(param_to_optimize, lr=1e-3)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10)

for epoch in range(cfg.num_epochs_stage1):
    manager.train_epoch_id(train_loader, optimizer, epoch, "train-stage-1")
    acc = evaluator.evaluate(manager.model)
    logger.log('[euclidean][global] R@1: %.4f | R@5: %.4f | R@10: %.4f' % (acc['top-1'], acc['top-5'], acc['top-10']))
    acc = cos_evaluator.evaluate(manager.model)
    logger.log('[cosine   ][global] R@1: %.4f | R@5: %.4f | R@10: %.4f' % (acc['top-1'], acc['top-5'], acc['top-10']))
    scheduler.step()
    manager.save_ckpt(epoch, acc, 'stage_1_id_last.pt')
manager.save_ckpt(epoch, acc, 'id_initialized.pt')

# Stage 2 (ID + Matching)
manager.melt_img_layer(num_layer_to_melt=cfg.image_melt_layer)
if cfg.np:
    param_to_optimize = build_graph_optimizer([manager.model, manager.id_cls, manager.rga_img_mlp, manager.rga_cap_mlp]) 
else:
    param_to_optimize = build_graph_optimizer([manager.model, manager.id_cls])    
optimizer = optim.Adam(param_to_optimize, lr=2e-4, weight_decay=1e-5)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10)

train_epoch = manager.train_epoch_regional if cfg.np else manager.train_epoch_global
for epoch in range(cfg.num_epochs_stage2):
    train_epoch(train_loader, optimizer, epoch, "train-stage-2")
    if cfg.np:
        acc = evaluator.evaluate(manager.model, manager.rga_img_mlp, manager.rga_cap_mlp)
        cos_acc = cos_evaluator.evaluate(manager.model, manager.rga_img_mlp, manager.rga_cap_mlp)
    else:
        acc = evaluator.evaluate(manager.model)
        cos_acc = cos_evaluator.evaluate(manager.model)
    logger.log('[euclidean][global] R@1: %.4f | R@5: %.4f | R@10: %.4f' % (acc['top-1'], acc['top-5'], acc['top-10']))
    logger.log('[cosine   ][global] R@1: %.4f | R@5: %.4f | R@10: %.4f' % (cos_acc['top-1'], cos_acc['top-5'], cos_acc['top-10']))
    scheduler.step()
    manager.save_ckpt(epoch, acc, 'stage_2_id_match_last.pt')


======== [Stage 1] ============


train-stage-1, epoch0: 9it [00:03,  2.87it/s]


KeyboardInterrupt: 